In [1]:
from Samplers.highd import *
from Samplers.plotting import *
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from scipy.stats import multivariate_normal, invwishart, dirichlet
from tqdm import tqdm
import matplotlib.patches as patches
import matplotlib.pyplot as plt

data = pd.read_csv('data/NHANES_adults_data_preprocessed.csv')


In [2]:
data

,Unnamed: 0,id_study,mid_year,age,sex,height,weight,bmi,waist,WHtR,...,samplewt_anthro,samplewt_hba1c,samplewt_bp,samplewt_chol,ethnicity,education,drug_hyper.1,self_mi,self_stroke,self_heart_failure
0,1,USA_1991_NHANES,1991,21,1,178.8,81.60,25.524376,88.8,0.496644,...,1737.579956,1737.579956,1523.000000,NaN,Hispanic,High School,0.0,0.0,0.0,0.0
1,2,USA_1991_NHANES,1991,32,2,162.2,61.65,23.433188,90.5,0.557953,...,1730.229980,1730.229980,1566.140015,NaN,Hispanic,University or College,0.0,0.0,0.0,0.0
2,3,USA_1991_NHANES,1991,48,2,156.9,68.05,27.642813,89.5,0.570427,...,19521.359375,19521.359375,18155.150391,52780.941406,Non-Hispanic White,University or College,0.0,0.0,0.0,0.0
3,4,USA_1991_NHANES,1991,35,1,177.4,92.50,29.392363,102.2,0.576099,...,28080.570312,28080.570312,22220.169922,NaN,Non-Hispanic White,University or College,0.0,0.0,0.0,0.0
4,5,USA_1991_NHANES,1991,48,1,168.2,70.60,24.954721,94.0,0.558859,...,1248.430054,1248.430054,1166.439941,2606.770020,Hispanic,Below High School,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
72106,121360,USA_2018_NHANES,2018,64,1,NaN,NaN,NaN,NaN,NaN,...,0.000000,0.000000,0.000000,0.000000,Non-Hispanic Black,High School,1.0,0.0,1.0,1.0
72107,121362,USA_2018_NHANES,2018,70,2,156.5,49.00,20.006329,82.2,0.525240,...,18338.711104,18338.711104,18338.711104,18338.711104,Other,High School,0.0,0.0,0.0,0.0
72108,121363,USA_2018_NHANES,2018,42,1,164.9,97.40,35.819345,114.8,0.696180,...,63661.951573,63661.951573,63661.951573,63661.951573,Hispanic,High School,0.0,0.0,0.0,0.0
72109,121364,USA_2018_NHANES,2018,41,2,162.6,69.10,26.135870,86.4,0.531365,...,17694.783346,17694.783346,17694.783346,17694.783346,Non-Hispanic Black,University or College,0.0,0.0,0.0,0.0


In [ ]:
# Select the columns of interest
columns = ['age', 'bmi']
data = data[columns]
data

,age,bmi
0,21,25.524376
1,32,23.433188
2,48,27.642813
3,35,29.392363
4,48,24.954721
...,...,...
72106,64,NaN
72107,70,20.006329
72108,42,35.819345
72109,41,26.135870


In [10]:
from scipy.spatial.distance import pdist

def h1(mu):
    return np.min(pdist(mu))

# def h2(mu):
#     """
#     Returns 1 if unique, 0 otherwise.
#     """
#     # Extract BMI and SBP values from mu
#     bmi = mu[:, 0]
#     sbp = mu[:, 1]
    
#     # Use np.digitize to find interval indices for BMI and SBP
#     bmi_blocks = np.digitize(bmi, bins=bmi_bounds)
#     sbp_blocks = np.digitize(sbp, bins=sbp_bounds)
    
#     # Combine BMI and SBP block indices into tuples
#     blocks = set(zip(bmi_blocks, sbp_blocks))
    
#     # If all blocks are unique, return 1; otherwise, return 0
#     return 1 if len(blocks) == len(mu) else 0


In [11]:
np.random.seed(42)
# Parameters
num_runs = 4
num_iterations = 10000
K = 4  # Number of clusters
X = data.to_numpy()

all_run_samples = []

for run in range(num_runs):
    print(f"Running Gibbs Sampler for Run {run + 1}")
    run_samples = bayesian_repulsive_randomwalk(X, K, num_iterations, h1, 0, 0.1)
    all_run_samples.append(run_samples)

from sklearn.decomposition import PCA
mu_samples = np.array([s[1] for s in run_samples])  # shape: (T, K, p)

# Use posterior mean after burn-in
posterior_mu = np.mean(mu_samples, axis=0)  # shape: (K, p)
pca = PCA(n_components=2)
mu_pca = pca.fit_transform(posterior_mu)

# Then plot in 2D:
plt.scatter(mu_pca[:, 0], mu_pca[:, 1])

Running Gibbs Sampler for Run 1


LinAlgError: SVD did not converge